<a href="https://colab.research.google.com/github/Maynul-haque/Diabetesdx/blob/main/pdfbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF -q
!pip install openai -q
!pip install gradio -q
!pip install bardapi>=0.1.8

In [ ]:
import fitz
import re
import numpy as np
import tensorflow_hub as hub
from bardapi import Bard
import os
from tqdm.auto import tqdm
from sklearn.neighbors import NearestNeighbors
import sklearn
sklearn.__version__

In [3]:
#upload file 
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
  file_path = '/content/' + filename
  print(f'File saved to {file_path}')

Saving Dopamine-Detox.pdf to Dopamine-Detox.pdf
File saved to /content/Dopamine-Detox.pdf


In [4]:
def preprocess(text):
    '''
    preprocess chunks
    1. Replace new line character with whitespace.
    2. Replace redundant whitespace with a single whitespace
    '''
    text = text.replace('\n', ' ')
    text = re.sub('\s+', ' ', text)
    return text

def pdf_to_text(file_path, start_page=1, end_page=None):
    '''
    convert PDF document to text
    '''
    doc = fitz.open(file_path)
    total_pages = doc.page_count

    if end_page is None:
        end_page = total_pages

    text_list = []

    for i in tqdm(range(start_page-1, end_page)):
        text = doc.load_page(i).get_text("text")
        text = preprocess(text)
        text_list.append(text)

    doc.close()
    return text_list

def text_to_chunks(texts, word_length=150, start_page=1):
    '''
    convert list of texts to smaller chunks of length `word_length`
    '''
    text_toks = [t.split(' ') for t in texts]
    page_nums = []
    chunks = []
    
    for idx, words in enumerate(text_toks):
        for i in range(0, len(words), word_length):
            chunk = words[i:i+word_length]
            if (i+word_length) > len(words) and (len(chunk) < word_length) and (
                len(text_toks) != (idx+1)):
                text_toks[idx+1] = chunk + text_toks[idx+1]
                continue
            chunk = ' '.join(chunk).strip()
            chunk = f'[{idx+start_page}]' + ' ' + '"' + chunk + '"'
            chunks.append(chunk)
    return chunks

In [5]:
class SemanticSearch:
    
    def __init__(self):
        self.use = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
        self.fitted = False
    
    def fit(self, data, batch=1000, n_neighbors=5):
        self.data = data
        self.embeddings = self.get_text_embedding(data, batch=batch)
        n_neighbors = min(n_neighbors, len(self.embeddings))
        self.nn = NearestNeighbors(n_neighbors=n_neighbors)
        self.nn.fit(self.embeddings)
        self.fitted = True
    
    def __call__(self, text, return_data=True):
        inp_emb = self.use([text])
        neighbors = self.nn.kneighbors(inp_emb, return_distance=False)[0]
        
        if return_data:
            return [self.data[i] for i in neighbors]
        else:
            return neighbors
    
    def get_text_embedding(self, texts, batch=1000):
        embeddings = []
        for i in tqdm(range(0, len(texts), batch)):
            text_batch = texts[i:(i+batch)]
            emb_batch = self.use(text_batch)
            embeddings.append(emb_batch)
        embeddings = np.vstack(embeddings)
        return embeddings

In [6]:
# Initialize the Bard API client
api = 'WghvVVNNvuGlxO9oi3bxO-6R3X0qP8ihrvnDUKjdn-az0eITlQJs6K9CEH9erOYgDeuKtg.'
bard = Bard(token=api)

recommender = SemanticSearch()

In [7]:
def load_recommender(file_path, start_page=1):
    global recommender
    texts = pdf_to_text(file_path, start_page=start_page)
    chunks = text_to_chunks(texts, start_page=start_page)
    recommender.fit(chunks)
    return 'Corpus Loaded.'

def generate_answer(question):
    topn_chunks = recommender(question)
    prompt = ""
    prompt += 'search results:\n\n'
    for c in topn_chunks:
        prompt += c + '\n\n'
        
    prompt += "Instructions: Compose a comprehensive reply to the query using the search results given."\
              "Cite each reference using [number] notation (every result has this number at the beginning)."\
              "Citation should be done at the end of each sentence. If the search results mention multiple subjects"\
              "with the same name, create separate answers for each. Only include information found in the results and"\
              "don't add any additional information. Make sure the answer is correct and don't output false content."\
              "If the text does not relate to the query, simply state 'Found Nothing'. Don't write 'Answer:'"\
              "Directly start the answer.\n"
    
    prompt += f"Query: {question}\n\n"
    answer = bard.get_answer(prompt)['content']
    return answer


In [9]:
# Call the load_recommender function to load the PDF and fit the recommender
pdf_file_path = 'path/to/your/pdf/file.pdf'  # Update with the actual file path
start_page_number = 1  # Specify the starting page number if needed
load_recommender(file_path, start_page=start_page_number)

# Call the generate_answer function to generate an answer for a question
question = "how to beat procastination?"  # Replace with your actual question
answer = generate_answer(question)
print(answer)


  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Here are some tips on how to beat procrastination, based on my knowledge:

* **Understand the psychology of procrastination.** Procrastination is often driven by fear of failure, perfectionism, or a lack of motivation. Once you understand the underlying reasons for your procrastination, you can start to develop strategies to overcome it.
* **Break down large tasks into smaller, more manageable steps.** This will make the task seem less daunting and more achievable.
* **Set deadlines for yourself and stick to them.** This will help you stay on track and avoid putting things off until the last minute.
* **Reward yourself for completing tasks.** This will help you stay motivated and make procrastination less appealing.
* **Avoid distractions.** This means turning off your phone, closing your email, and finding a quiet place to work.
* **Take breaks.** Don't try to work for hours on end without taking a break. Get up and move around, or step outside for some fresh air.
* **Don't be afraid 